In [14]:
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout
import pandas as pd

In [ ]:
os.chdir("G:\Mon Drive\M2\PGM\Wikipedia-Clustering")

G:\Mon Drive\M2\PGM\Wikipedia-Clustering


In [25]:
graph_df = pd.read_csv("data/small_graphs/squirrel/musae_squirrel_edges.csv")
names = pd.read_csv('data/wiki-topcats-page-names.txt.gz', compression='gzip', header=None, sep='\t', quotechar='"')

G = nx.from_pandas_edgelist(graph_df, source="id1", target="id2")

In [53]:
print(G)
nodes = G.nodes()
print(type(list(nodes)))

Graph with 5201 nodes and 198493 edges
<class 'list'>


In [61]:
nodes = list(nodes)
print([[node] for node in nodes])

[[3475], [2849], [3106], [808], [4555], [3563], [1527], [3327], [402], [4066], [3908], [2820], [4903], [715], [5112], [817], [4855], [5057], [4481], [3975], [2408], [2539], [4236], [3372], [4303], [4658], [3829], [5078], [313], [4410], [777], [3609], [5122], [4650], [5093], [4862], [2930], [169], [5067], [3700], [2238], [2890], [4544], [2371], [5123], [1481], [1039], [3668], [1654], [1883], [4796], [2173], [3644], [4664], [1316], [4965], [5194], [2219], [3630], [3603], [3253], [3413], [4989], [2972], [5074], [2958], [3691], [4419], [5125], [3909], [4346], [5196], [4365], [3852], [4847], [3730], [3891], [4434], [4730], [4758], [792], [4598], [5050], [5115], [4088], [4471], [3310], [5121], [514], [3078], [5126], [4616], [1548], [4621], [5132], [4625], [4627], [2068], [4629], [5142], [4120], [5145], [5146], [2040], [5148], [4638], [5150], [4640], [4128], [4129], [5160], [2089], [3115], [5164], [5165], [5169], [5174], [2103], [4665], [4158], [1087], [4160], [5186], [5189], [3142], [3143], 

In [ ]:
def upper_bound_check(n_edges_in, n_edges_out):
    return (n_edges_in>=1 and n_edges_out==0)

def bottom_up_clustering(G, limit = 50): # Based on n_edges_in/n_edges_out criterion
    A = nx.adjacency_matrix(G).toarray()
    cluster_adjacency = A.copy()
    cluster_degrees = np.sum(cluster_adjacency, axis=1)
    clusters = [[node] for node in list(G.nodes())]
    n = A.shape[0]


    for k in range(n, 0, -1):
        print("Iteration", n-k)
        best_possible_found = False

        degree_argsort = np.argsort(cluster_degrees) # Starting with smallest degrees heuristic
        sorted_degrees = cluster_degrees[degree_argsort]
        first_non_null_degree_index = np.argmax(sorted_degrees>0.1)

        if k==limit or first_non_null_degree_index == k-1:
            return clusters

        max_criterion = 0

        for i in range(first_non_null_degree_index, k):
            c1 = degree_argsort[i] # c1 is the cluster with the smallest degree
            

            for j in range(i + 1, k):
                c2 = degree_argsort[k-1-j] # c2 is the cluster with the biggest degree

                n_edges_in = cluster_adjacency[c1, c2]
                if not n_edges_in:
                    continue
                n_edges_out = cluster_degrees[c1] + cluster_degrees[c2] - 2*n_edges_in
                n_out = max(n_edges_out, .0001)

                criterion = n_edges_in / n_out
                if criterion > max_criterion:
                    max_criterion = criterion
                    best_n_edges_out = n_edges_out
                    best_c1 = c1
                    best_c2 = c2

                if upper_bound_check(n_edges_in, n_edges_out):
                    best_possible_found=True
                    print("Upper bound triggered")
                    break
            if best_possible_found:
                break
        
        #merge clusters
        clusters[best_c1] = [clusters[best_c1], clusters[best_c2]]


        cluster_adjacency[best_c1] += cluster_adjacency[best_c2]
        cluster_adjacency[:, best_c1] = cluster_adjacency[best_c1]


        # The idea is that to not have to delete the row and column of the merged cluster, we just copy the last row and column to the row and column of the merged cluster
        # We then only consider the first k*k entries of the cluster adjacency matrix for future iterations
        cluster_adjacency[best_c2] = cluster_adjacency[k-1]
        cluster_adjacency[:,best_c2] = cluster_adjacency[k-1]
        cluster_adjacency[best_c1, best_c2] = cluster_adjacency[best_c1, k-1] + cluster_adjacency[best_c2, k-1]

        cluster_degrees[best_c1] = best_n_edges_out
        previously_connected_to_both = cluster_adjacency[best_c1,:k] * cluster_adjacency[best_c2,:k]
        cluster_degrees[previously_connected_to_both] -= 1
    
    return clusters

In [ ]:
clusters = bottom_up_clustering(G)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [34]:
for node in nodes:
        print(names[0][node])

3475 Diana Zuckerman
2849 Middle meningeal artery
3106 Auditory fatigue
808 Micrasta
4555 Methanofuran
3563 RUNX2
1527 Tears (X Japan song)
3327 Enucleation (microbiology)
402 Notobubastes
4066 Orgel's rule
3908 Euphorbia palmeri
2820 Corrugator supercilii muscle
4903 Postsynaptic potential
715 Cordillerita
5112 Crystallin
817 Neomastogenius
4855 ErdheimChester disease
5057 Operator (biology)
4481 Glucosylceramidase
3975 Euphorbia kuwaleana
2408 Anterior nuclei of thalamus
2539 Clobenpropit
4236 Formyltetrahydrofolate dehydrogenase
3372 Rheobase
4303 Azobenzene reductase
4658 Cidofovir
3829 Euphorbia zakamenae
5078 P element
313 Chalcophora
4410 ATP adenylyltransferase
777 Paraancylotela
3609 Capitate bone
5122 Assisted reproductive technology
4650 Abacavir
5093 Personal genomics
4862 Paroxysmal depolarizing shift
2930 Facial muscles
169 Sarver, Pennsylvania
5067 Prophage
3700 Patellar reflex
2238 Day care sex abuse hysteria
2890 Mylohyoid line
4544 6-pyruvoyltetrahydropterin synthase
